In [1]:
import pandas as pd
import numpy as np
import itertools

from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *
from imblearn.metrics import *

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

from sklearn.neural_network import MLPClassifier


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import matplotlib.pyplot as plt

In [2]:
data= pd.read_csv('mendeley_18.csv', header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,3,1,5,72,0,0,0,0,0,0,...,0,0,1,1,0,1,1,-1,1,1
1,3,1,3,144,0,0,0,0,2,0,...,0,0,0,1,-1,1,1,1,1,1
2,3,1,2,58,0,0,0,0,0,0,...,0,0,0,1,0,-1,1,-1,0,1
3,3,1,6,79,1,0,0,0,0,0,...,0,0,0,1,-1,1,1,1,-1,1
4,3,0,4,46,0,0,0,0,0,0,...,1,0,0,1,1,-1,0,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3,1,1,50,0,0,0,0,0,0,...,0,0,0,1,1,-1,1,0,1,0
9996,2,1,4,59,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,1,0
9997,2,1,4,57,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,1,0
9998,3,1,1,49,0,0,0,0,0,0,...,1,0,0,1,1,0,1,0,1,0


# train-test split

In [3]:
x= data.drop([48],axis=1)
x

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,3,1,5,72,0,0,0,0,0,0,...,0,0,0,1,1,0,1,1,-1,1
1,3,1,3,144,0,0,0,0,2,0,...,0,0,0,0,1,-1,1,1,1,1
2,3,1,2,58,0,0,0,0,0,0,...,0,0,0,0,1,0,-1,1,-1,0
3,3,1,6,79,1,0,0,0,0,0,...,1,0,0,0,1,-1,1,1,1,-1
4,3,0,4,46,0,0,0,0,0,0,...,0,1,0,0,1,1,-1,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3,1,1,50,0,0,0,0,0,0,...,0,0,0,0,1,1,-1,1,0,1
9996,2,1,4,59,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,1
9997,2,1,4,57,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,1
9998,3,1,1,49,0,0,0,0,0,0,...,0,1,0,0,1,1,0,1,0,1


In [4]:
y=data[48]
y

0       1
1       1
2       1
3       1
4       1
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: 48, Length: 10000, dtype: int64

In [5]:
x_train=x
y_train=y

# defining models

In [6]:
knn= KNeighborsClassifier(n_neighbors=10,n_jobs=-1)
svm= SVC(random_state=10,kernel='rbf')
lr = LogisticRegression(max_iter = 300)

rf= RandomForestClassifier(n_jobs=-1,random_state=10)
ada= AdaBoostClassifier(random_state=100)
extra = ExtraTreesClassifier(n_jobs=-1,random_state=10)

lgbm = LGBMClassifier(n_jobs=-1,random_state=10)
grad = GradientBoostingClassifier()
xg = XGBClassifier(use_label_encoder =False, eval_metric='logloss')

nn= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (32,32,32), random_state=10, max_iter=500)

In [7]:
classifiers= {
    "RandomForest": rf,
    "ExtraTree": extra,
    "gradBoost": grad,
    "adaboost": ada,
    "LGBM": lgbm,
    "xgb": xg,
    "nn":nn}

In [8]:
classifier_2 = nn

In [9]:
scores={'accuracy': make_scorer(accuracy_score),
        'recall' : make_scorer(recall_score),
        'precision':make_scorer(precision_score),
        'gmean': make_scorer(geometric_mean_score),
        'f1':make_scorer(f1_score),
        'roc': make_scorer(roc_auc_score)
        }

# training

In [12]:
models_scores_results_10, models_names_10 = list(), list() 

In [15]:
rf_result = cross_validate(rf, x_train, y_train, cv = 10, scoring=scores,n_jobs=-1)
df= pd.DataFrame(rf_result)
df.mean(axis=0)

fit_time          0.442173
score_time        0.043752
test_accuracy     0.977200
test_recall       0.972000
test_precision    0.982263
test_gmean        0.977172
test_f1           0.977077
test_roc          0.977200
dtype: float64

In [16]:
ada_result= cross_validate(ada, x_train, y_train, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(ada_result)
df.mean(axis=0)

fit_time          0.530940
score_time        0.014571
test_accuracy     0.967600
test_recall       0.972000
test_precision    0.963697
test_gmean        0.967558
test_f1           0.967772
test_roc          0.967600
dtype: float64

In [17]:
extra_result= cross_validate(extra, x_train, y_train, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(extra_result)
df.mean(axis=0)

fit_time          0.401826
score_time        0.134421
test_accuracy     0.973600
test_recall       0.960400
test_precision    0.986490
test_gmean        0.973475
test_f1           0.973201
test_roc          0.973600
dtype: float64

In [18]:
grad_result= cross_validate(grad, x_train, y_train, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(grad_result)
df.mean(axis=0)

fit_time          1.705094
score_time        0.006250
test_accuracy     0.972500
test_recall       0.972200
test_precision    0.972803
test_gmean        0.972487
test_f1           0.972477
test_roc          0.972500
dtype: float64

In [19]:
lgbm_result= cross_validate(lgbm, x_train, y_train, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(lgbm_result)
df.mean(axis=0)

fit_time          0.211191
score_time        0.015865
test_accuracy     0.980200
test_recall       0.977200
test_precision    0.983191
test_gmean        0.980177
test_f1           0.980151
test_roc          0.980200
dtype: float64

In [20]:
xg_result= cross_validate(xg, x_train, y, cv = 10, scoring=scores)
df= pd.DataFrame(xg_result)
df.mean(axis=0)

fit_time          0.472583
score_time        0.010514
test_accuracy     0.981500
test_recall       0.978400
test_precision    0.984614
test_gmean        0.981470
test_f1           0.981449
test_roc          0.981500
dtype: float64

In [23]:
nn_result= cross_validate(nn, x_train, y, cv = 10, scoring=scores, n_jobs=-1)
df= pd.DataFrame(nn_result)
df.mean(axis=0)

fit_time          11.366709
score_time         0.004688
test_accuracy      0.953600
test_recall        0.951600
test_precision     0.955646
test_gmean         0.953503
test_f1            0.953448
test_roc           0.953600
dtype: float64

In [11]:
svm_result= cross_validate(svm, x_train, y, cv = 10, scoring=scores)
df= pd.DataFrame(svm_result)
df.mean(axis=0)

fit_time          1.919970
score_time        1.099316
test_accuracy     0.836800
test_recall       0.905800
test_precision    0.810829
test_gmean        0.827696
test_f1           0.850772
test_roc          0.836800
dtype: float64

In [12]:
knn_result= cross_validate(knn, x_train, y, cv = 10, scoring=scores)
df= pd.DataFrame(knn_result)
df.mean(axis=0)

fit_time          0.005601
score_time        0.236455
test_accuracy     0.833500
test_recall       0.860400
test_precision    0.823507
test_gmean        0.830606
test_f1           0.838798
test_roc          0.833500
dtype: float64

In [19]:
lr = LogisticRegression(max_iter = 1500, solver = 'liblinear')
lr_result= cross_validate(lr, x_train, y, cv = 10, scoring=scores)
df= pd.DataFrame(lr_result)
df.mean(axis=0)

fit_time          0.059090
score_time        0.003401
test_accuracy     0.936600
test_recall       0.944200
test_precision    0.930654
test_gmean        0.936451
test_f1           0.937199
test_roc          0.936600
dtype: float64

In [14]:
nb= GaussianNB()
nb_result= cross_validate(nb, x_train, y, cv = 10, scoring=scores)
df= pd.DataFrame(nb_result)
df.mean(axis=0)

fit_time          0.007502
score_time        0.004575
test_accuracy     0.842600
test_recall       0.741600
test_precision    0.929105
test_gmean        0.836099
test_f1           0.824122
test_roc          0.842600
dtype: float64

# Stacking - probability based

In [24]:
# meta - nn,  cv - 10


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (6,12,24,12,6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg, nn]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [25]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          64.271818
score_time         0.113592
test_accuracy      0.981300
test_recall        0.977400
test_precision     0.985226
test_gmean         0.981253
test_f1            0.981220
test_roc           0.981300
dtype: float64

In [26]:
# meta - nn,  cv - 10
# without nn as base classifier


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (6,12,24,12,6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [27]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          22.763085
score_time         0.143905
test_accuracy      0.981900
test_recall        0.977000
test_precision     0.986772
test_gmean         0.981865
test_f1            0.981818
test_roc           0.981900
dtype: float64

# stacking - optimize meta classifier

In [28]:
sel_classifier= [rf, ada, extra, lgbm, grad, xg]

nn_basic = MLPClassifier(random_state=10,verbose = True)

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_basic, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

params= {
    'meta_classifier__hidden_layer_sizes': [(12,12,12), (24,32,24), (6,6,6),(6,12,6),(12,6,6),(6,12,32,12,6)],
}

grid = GridSearchCV(estimator=scl, 
                    param_grid=params, 
                    cv=10,
                    refit=True,
                    n_jobs=-1)

grid.fit(x_train, y_train)


Iteration 1, loss = 0.61714193
Iteration 2, loss = 0.36821330
Iteration 3, loss = 0.14614996
Iteration 4, loss = 0.07222038
Iteration 5, loss = 0.05842353
Iteration 6, loss = 0.05565146
Iteration 7, loss = 0.05452441
Iteration 8, loss = 0.05414254
Iteration 9, loss = 0.05292767
Iteration 10, loss = 0.05207144
Iteration 11, loss = 0.05132113
Iteration 12, loss = 0.05082327
Iteration 13, loss = 0.05068811
Iteration 14, loss = 0.05045275
Iteration 15, loss = 0.05038991
Iteration 16, loss = 0.04973777
Iteration 17, loss = 0.04952718
Iteration 18, loss = 0.04936658
Iteration 19, loss = 0.04927813
Iteration 20, loss = 0.04940434
Iteration 21, loss = 0.04838373
Iteration 22, loss = 0.04857156
Iteration 23, loss = 0.04905878
Iteration 24, loss = 0.04809038
Iteration 25, loss = 0.04822909
Iteration 26, loss = 0.04835042
Iteration 27, loss = 0.04816193
Iteration 28, loss = 0.04759887
Iteration 29, loss = 0.04779576
Iteration 30, loss = 0.04744891
Iteration 31, loss = 0.04701798
Iteration 32, los

GridSearchCV(cv=10,
             estimator=StackingCVClassifier(classifiers=[RandomForestClassifier(n_jobs=-1,
                                                                                random_state=10),
                                                         AdaBoostClassifier(random_state=100),
                                                         ExtraTreesClassifier(n_jobs=-1,
                                                                              random_state=10),
                                                         LGBMClassifier(random_state=10),
                                                         GradientBoostingClassifier(),
                                                         XGBClassifier(base_score=None,
                                                                       booster=None,
                                                                       colsample_bylevel=None,
                                                                  

In [31]:
df= pd.DataFrame(grid.cv_results_)
df.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_meta_classifier__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
5,36.324671,2.584236,0.082819,0.018625,"(6, 12, 32, 12, 6)","{'meta_classifier__hidden_layer_sizes': (6, 12...",0.981,0.982,0.989,0.979,0.988,0.995,0.980,0.971,0.971,0.983,0.9819,0.007148,1
2,40.415400,4.147627,0.241525,0.189733,"(6, 6, 6)","{'meta_classifier__hidden_layer_sizes': (6, 6,...",0.978,0.979,0.987,0.981,0.986,0.993,0.980,0.972,0.976,0.984,0.9816,0.005748,2
4,42.355330,2.994399,0.111014,0.019143,"(12, 6, 6)","{'meta_classifier__hidden_layer_sizes': (12, 6...",0.980,0.980,0.989,0.978,0.986,0.993,0.980,0.972,0.974,0.982,0.9814,0.006119,3
0,39.098017,5.967265,0.204212,0.113749,"(12, 12, 12)","{'meta_classifier__hidden_layer_sizes': (12, 1...",0.978,0.981,0.989,0.978,0.986,0.992,0.982,0.971,0.972,0.983,0.9812,0.006431,4
3,42.039618,2.577155,0.107600,0.011377,"(6, 12, 6)","{'meta_classifier__hidden_layer_sizes': (6, 12...",0.978,0.979,0.988,0.979,0.986,0.992,0.979,0.971,0.973,0.984,0.9809,0.006236,5
1,44.404492,5.525419,0.179370,0.139294,"(24, 32, 24)","{'meta_classifier__hidden_layer_sizes': (24, 3...",0.978,0.978,0.988,0.983,0.987,0.995,0.973,0.970,0.972,0.982,0.9806,0.007539,6


# feature selection

In [32]:
from sklearn.feature_selection import RFECV

In [33]:
selector = RFECV(rf, cv=5, n_jobs=-1)
selector = selector.fit(x, y)
rf_col,= np.where(selector.ranking_==1)
rf_col

array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47], dtype=int64)

In [34]:
selector = RFECV(ada, cv=5, n_jobs=-1)
selector = selector.fit(x, y)
ada_col,= np.where(selector.ranking_==1)
ada_col

array([ 0,  2,  3,  4,  5,  9, 10, 13, 14, 20, 21, 22, 24, 26, 27, 28, 29,
       30, 33, 34, 38, 39, 42, 46, 47], dtype=int64)

In [35]:
selector = RFECV(extra, cv=5, n_jobs=-1)
selector = selector.fit(x, y)
extra_col= np.where(selector.ranking_==1)
extra_col

(array([ 0,  2,  3,  4, 10, 13, 20, 21, 22, 24, 26, 27, 28, 29, 33, 34, 38,
        39, 44, 46, 47], dtype=int64),)

In [36]:
selector = RFECV(lgbm, cv=5, n_jobs=-1)
selector = selector.fit(x, y)
lgbm_col= np.where(selector.ranking_==1)
lgbm_col

(array([ 0,  1,  2,  3,  4,  5,  8,  9, 10, 11, 13, 14, 15, 18, 20, 21, 22,
        24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 38, 39, 40, 41, 44, 45, 46,
        47], dtype=int64),)

In [37]:
selector = RFECV(grad, cv=5, n_jobs=-1)
selector = selector.fit(x, y)
grad_col= np.where(selector.ranking_==1)
grad_col

(array([ 0,  2,  3,  4,  5,  8,  9, 10, 13, 14, 16, 21, 22, 24, 26, 27, 28,
        29, 33, 34, 38, 39, 45, 46, 47], dtype=int64),)

In [38]:
selector = RFECV(xg, cv=5, n_jobs=-1)
selector = selector.fit(x, y)
xg_col= np.where(selector.ranking_==1)
xg_col

(array([ 0,  2,  4,  5,  8,  9, 10, 11, 13, 14, 22, 24, 25, 26, 27, 28, 29,
        31, 32, 33, 34, 38, 39, 40, 45, 46, 47], dtype=int64),)

# stacking with subsets of features

In [39]:
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline

In [40]:
pipe_rf = make_pipeline(ColumnSelector(cols=(0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47)),rf)

In [41]:
pipe_ada = make_pipeline(ColumnSelector(cols=(0,  2,  3,  4,  5,  9, 10, 13, 14, 20, 21, 22, 24, 26, 27, 28, 29,
       30, 33, 34, 38, 39, 42, 46, 47)),ada)

In [42]:
pipe_extra = make_pipeline(ColumnSelector(cols=(0,  2,  3,  4, 10, 13, 20, 21, 22, 24, 26, 27, 28, 29, 33, 34, 38,
        39, 44, 46, 47)),extra)

In [43]:
pipe_lgbm = make_pipeline(ColumnSelector(cols=(0,  1,  2,  3,  4,  5,  8,  9, 10, 11, 13, 14, 15, 18, 20, 21, 22,
        24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 38, 39, 40, 41, 44, 45, 46,
        47)),lgbm)

In [44]:
pipe_grad = make_pipeline(ColumnSelector(cols=(0,  2,  3,  4,  5,  8,  9, 10, 13, 14, 16, 21, 22, 24, 26, 27, 28,
        29, 33, 34, 38, 39, 45, 46, 47)),grad)

In [45]:
pipe_xg = make_pipeline(ColumnSelector(cols=(0,  2,  4,  5,  8,  9, 10, 11, 13, 14, 22, 24, 25, 26, 27, 28, 29,
        31, 32, 33, 34, 38, 39, 40, 45, 46, 47)),xg)

In [46]:
nn= MLPClassifier(activation = "relu", hidden_layer_sizes = (6, 12, 32, 12, 6), random_state=10)

sclf = StackingCVClassifier(classifiers=[pipe_rf, pipe_ada, pipe_extra, pipe_lgbm, pipe_grad, pipe_xg], 
                            meta_classifier=nn,
                            use_probas = True,
                            random_state=42)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)

In [47]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          19.378408
score_time         0.082739
test_accuracy      0.980900
test_recall        0.976400
test_precision     0.985327
test_gmean         0.980875
test_f1            0.980814
test_roc           0.980900
dtype: float64

In [48]:
sel_classifier= [pipe_rf, pipe_ada, pipe_extra, pipe_lgbm, pipe_grad, pipe_xg]

nn = MLPClassifier(random_state=10,verbose = True)

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

params= {
    'meta_classifier__hidden_layer_sizes': [(24,32,24), (6,6,6),(12,6,6),(6,12,32,12,6),(6,12,24,36,24,12,6)],
}

grid = GridSearchCV(estimator=scl, 
                    param_grid=params, 
                    cv=10,
                    refit=True,
                    n_jobs=-1)

grid.fit(x_train, y_train)


Iteration 1, loss = 0.71568247
Iteration 2, loss = 0.49040722
Iteration 3, loss = 0.25149136
Iteration 4, loss = 0.12221053
Iteration 5, loss = 0.07922964
Iteration 6, loss = 0.06550729
Iteration 7, loss = 0.06022840
Iteration 8, loss = 0.05808860
Iteration 9, loss = 0.05682885
Iteration 10, loss = 0.05610499
Iteration 11, loss = 0.05562725
Iteration 12, loss = 0.05515154
Iteration 13, loss = 0.05486390
Iteration 14, loss = 0.05460022
Iteration 15, loss = 0.05444276
Iteration 16, loss = 0.05417749
Iteration 17, loss = 0.05395469
Iteration 18, loss = 0.05369328
Iteration 19, loss = 0.05358234
Iteration 20, loss = 0.05336230
Iteration 21, loss = 0.05325823
Iteration 22, loss = 0.05309899
Iteration 23, loss = 0.05300421
Iteration 24, loss = 0.05288205
Iteration 25, loss = 0.05294038
Iteration 26, loss = 0.05278838
Iteration 27, loss = 0.05267996
Iteration 28, loss = 0.05261776
Iteration 29, loss = 0.05230492
Iteration 30, loss = 0.05237170
Iteration 31, loss = 0.05198887
Iteration 32, los

GridSearchCV(cv=10,
             estimator=StackingCVClassifier(classifiers=[Pipeline(steps=[('columnselector',
                                                                          ColumnSelector(cols=(0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5,
                                                                                               7,
                                                                                               8,
                                                                                               9,
      

In [49]:
df= pd.DataFrame(grid.cv_results_)
df.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_meta_classifier__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
2,38.844759,2.789992,0.182825,0.166115,"(12, 6, 6)","{'meta_classifier__hidden_layer_sizes': (12, 6...",0.985,0.978,0.990,0.980,0.983,0.992,0.979,0.973,0.980,0.983,0.9823,0.005367,1
3,40.501064,1.640083,0.093757,0.009883,"(6, 12, 32, 12, 6)","{'meta_classifier__hidden_layer_sizes': (6, 12...",0.984,0.978,0.991,0.980,0.979,0.992,0.980,0.977,0.977,0.984,0.9822,0.005212,2
1,39.941713,3.209237,0.104037,0.018282,"(6, 6, 6)","{'meta_classifier__hidden_layer_sizes': (6, 6,...",0.984,0.979,0.987,0.978,0.985,0.992,0.982,0.969,0.979,0.984,0.9819,0.005873,3
4,35.771682,5.721634,0.071899,0.017410,"(6, 12, 24, 36, 24, 12, 6)","{'meta_classifier__hidden_layer_sizes': (6, 12...",0.988,0.980,0.989,0.978,0.983,0.990,0.976,0.973,0.979,0.983,0.9819,0.005449,3
0,28.187922,10.562209,0.123499,0.064328,"(24, 32, 24)","{'meta_classifier__hidden_layer_sizes': (24, 3...",0.983,0.979,0.991,0.980,0.982,0.992,0.969,0.974,0.980,0.984,0.9814,0.006576,5


In [50]:
# meta - nn,  cv - 10
# without nn as base classifier


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (12,6,6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [51]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          23.076894
score_time         0.082759
test_accuracy      0.981700
test_recall        0.977800
test_precision     0.985564
test_gmean         0.981676
test_f1            0.981636
test_roc           0.981700
dtype: float64

In [52]:
# meta - nn,  cv - 10
# without nn as base classifier


nn_meta= MLPClassifier(activation = "relu", alpha = 0.001, hidden_layer_sizes = (6, 12, 32, 12, 6), random_state=10)

sel_classifier= [rf, ada, extra, lgbm, grad, xg]

scl = StackingCVClassifier(classifiers= sel_classifier,
                            meta_classifier = nn_meta, # use meta-classifier
                            use_probas = True,   # use_probas = True/False
                            random_state = 10)

st_result= cross_validate(scl, x_train, y, cv = 10, scoring=scores,n_jobs=-1)


In [53]:
df= pd.DataFrame(st_result)
df.mean(axis=0)

fit_time          22.250925
score_time         0.089071
test_accuracy      0.981500
test_recall        0.975400
test_precision     0.987505
test_gmean         0.981461
test_f1            0.981375
test_roc           0.981500
dtype: float64